# Introduction

### The goal of this project is to fine-tune a pre-existing model to use to get the top 5 answers for a question from a customer

In [1]:
!pip install ipywidgets

In [2]:
!pip install pandas

In [3]:
!pip install wordcloud

#### Importing Modules

In [4]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import (GPT2Config,GPT2LMHeadModel,GPT2Tokenizer)
import torch
from string import punctuation as pnc
from collections import Counter
from scipy import spatial
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import torch
import pylab as pl
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from multiprocessing import Pool

In [5]:
# function to determine if columns in file have null values
def get_percent_of_na(df, num):
    count = 0
    df = df.copy()
    s = (df.isna().sum() / df.shape[0])
    for column, percent in zip(s.index, s.values):
        num_of_nulls = df[column].isna().sum()
        if num_of_nulls == 0:
            continue
        else:
            count += 1
        print('Column {} has {:.{}%} percent of Nulls, and {} of nulls'.format(column, percent, num, num_of_nulls))
    if count != 0:
        print("\033[1m" + 'There are {} columns with NA.'.format(count) + "\033[0m")
    else:
        print()
        print("\033[1m" + 'There are no columns with NA.' + "\033[0m")
        
# function to display general information about the dataset
def get_info(df):
    """
    This function uses the head(), info(), describe(), shape() and duplicated() 
    methods to display the general information about the dataset.
    """
    print("\033[1m" + '-'*100 + "\033[0m")
    print('Head:')
    print()
    display(df.head())
    print('-'*100)
    print('Info:')
    print()
    display(df.info())
    print('-'*100)
    print('Describe:')
    print()
    display(df.describe())
    print('-'*100)
    display(df.describe)
    print()
    print('Columns with nulls:')
    display(get_percent_of_na(df, 4))  # check this out
    print('-'*100)
    print('Shape:')
    print(df.shape)
    print('-'*100)
    print('Duplicated:')
    print("\033[1m" + 'We have {} duplicated rows.\n'.format(df.duplicated().sum()) + "\033[0m")

In [6]:
def clean_paragraph(paragraph):
    # Remove HTML tags using BeautifulSoup
    cleaned_text = BeautifulSoup(paragraph, 'html.parser').get_text()

    # Remove extra whitespaces and newline characters
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

def clear_text(text):
    
    pattern = r"[^a-zA-Z']"
    text = re.sub(pattern, ' ', text)
    text = text.split()
    text = ' '.join(text)
    return text

def lemmatize(text):

    doc = nlp(text.lower())
    
    lemmas = []
    for token in doc:
        lemmas.append(token.lemma_)
        
    return ' '.join(lemmas)

def clean_text(text):
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"<.*?>", "", text)  # Modified regex to remove all HTML tags
    text = re.sub(r"\r\n|\n|\t", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    doc = nlp(text)
    words = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(words)

Importing and viewing the Questions Dataframe

In [7]:
questions = pd.read_csv("Questions.csv", encoding = "ISO-8859-1")
print(f'questiondf_length: {len(questions)}')
questions.head()


questiondf_length: 607282


,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [8]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607282 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            607282 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  607282 non-null  object 
 3   Score         607282 non-null  int64  
 4   Title         607282 non-null  object 
 5   Body          607282 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 27.8+ MB


Converting CreationDate to DateTime

In [9]:
questions["CreationDate"] = pd.to_datetime(questions["CreationDate"])

Viewing the Null values in the Questions Dataframe

In [10]:
questions.isna().sum()

Id                 0
OwnerUserId     6212
CreationDate       0
Score              0
Title              0
Body               0
dtype: int64

Dropping the null values in the dataset since they are just over 1% of our dataset. Therefore, it won't affect the dataframe enough to significantly skew our model. Once dropped, we can see which OwnerUserId goes with which OwnerUserID in the Answers Dataframe.

In [11]:
questions = questions.dropna().reset_index(drop=True)

Sanity Check

In [12]:
questions.isna().sum()

Id              0
OwnerUserId     0
CreationDate    0
Score           0
Title           0
Body            0
dtype: int64

In [13]:
questions = questions.drop_duplicates().reset_index(drop=True)

In [14]:
len(questions)

601070

In [15]:
questions["cleaned_questions"] = questions['Body'].apply(clean_paragraph)

In [16]:
questions["cleaned_title"] = questions['Title'].apply(clean_paragraph)

C:\Users\15135\AppData\Local\Temp\ipykernel_27556\1815705739.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned_text = BeautifulSoup(paragraph, 'html.parser').get_text()
C:\Users\15135\AppData\Local\Temp\ipykernel_27556\1815705739.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  cleaned_text = BeautifulSoup(paragraph, 'html.parser').get_text()


In [17]:
questions['cleaned_questions'][1]

'I have a cross-platform (Python) application which needs to generate a JPEG preview of the first page of a PDF. On the Mac I am spawning sips. Is there something similarly simple I can do on Windows?'

In [18]:
questions['cleaned_title'][1]

'Get a preview JPEG of a PDF on Windows?'

Importing and viewing the answers dataframe

In [19]:
answers = pd.read_csv("Answers.csv", encoding = "ISO-8859-1")
print(f'answerdf_length: {len(answers)}')
answers.head()


answerdf_length: 987122


,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50.0,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153.0,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161.0,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156.0,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157.0,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."


In [20]:
answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987122 entries, 0 to 987121
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            987122 non-null  int64  
 1   OwnerUserId   981755 non-null  float64
 2   CreationDate  987122 non-null  object 
 3   ParentId      987122 non-null  int64  
 4   Score         987122 non-null  int64  
 5   Body          987122 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 45.2+ MB


In [21]:
answers["CreationDate"] = pd.to_datetime(answers["CreationDate"])

Viewing the Null values in the Questions Dataframe

In [22]:
answers.isna().sum().reset_index(drop=True)

0       0
1    5367
2       0
3       0
4       0
5       0
dtype: int64

Dropping the null values in the dataset since they are under 1% of our dataset. Therefore, it won't affect the dataframe enough to significantly skew our model. 

In [23]:
answers_no_nan = answers.dropna().reset_index(drop=True)

Sanity Check

In [24]:
answers.isna().sum()

Id                 0
OwnerUserId     5367
CreationDate       0
ParentId           0
Score              0
Body               0
dtype: int64

In [25]:
answers = answers.drop_duplicates().reset_index(drop=True)

In [26]:
answers["cleaned_answers"] = answers['Body'].apply(clean_paragraph)

In [27]:
answers['cleaned_answers'][1]

"I haven't been able to find anything that does this directly. I think you'll have to iterate through the various font folders on the system: /System/Library/Fonts, /Library/Fonts, and there can probably be a user-level directory as well ~/Library/Fonts."

Importing and viewing the tags dataframe

In [28]:
tags = pd.read_csv("Tags.csv", encoding = "ISO-8859-1")
print(f'tagsdf_length: {len(tags)}')
tags.head()

tagsdf_length: 1885078


,Id,Tag
0,469,python
1,469,osx
2,469,fonts
3,469,photoshop
4,502,python


In [29]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885078 entries, 0 to 1885077
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 28.8+ MB


Viewing the Null values in the Tags Dataframe

In [30]:
tags.isna().sum()

Id       0
Tag    443
dtype: int64

Dropping the null values in the dataset since they are just over 1% of our dataset. Therefore, it won't affect the dataframe enough to significantly skew our model. 

In [31]:
tags = tags.dropna().reset_index(drop=True)

Sanity Check

In [32]:
tags.isna().sum()

Id     0
Tag    0
dtype: int64

In [33]:
tags = tags.drop_duplicates().reset_index(drop=True)

In [34]:
tags['Tag'] = tags['Tag'].apply(clean_paragraph)

C:\Users\15135\AppData\Local\Temp\ipykernel_27556\1815705739.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  cleaned_text = BeautifulSoup(paragraph, 'html.parser').get_text()


In [35]:
tags.head()

,Id,Tag
0,469,python
1,469,osx
2,469,fonts
3,469,photoshop
4,502,python


In [36]:
merged_df = questions.merge(answers, how='left', left_on='Id', right_on='ParentId', suffixes=("_question", "_answer"))

In [37]:
merged_df.head(10)

,Id_question,OwnerUserId_question,CreationDate_question,Score_question,Title,Body_question,cleaned_questions,cleaned_title,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer,cleaned_answers
0,469,147.0,2008-08-02 15:11:16+00:00,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,I am using the Photoshop's javascript API to f...,How can I find the full path to a font from it...,497.0,50.0,2008-08-02 16:56:53+00:00,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,open up a terminal (Applications->Utilities->T...
1,469,147.0,2008-08-02 15:11:16+00:00,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,I am using the Photoshop's javascript API to f...,How can I find the full path to a font from it...,518.0,153.0,2008-08-02 17:42:28+00:00,469.0,2.0,<p>I haven't been able to find anything that d...,I haven't been able to find anything that does...
2,469,147.0,2008-08-02 15:11:16+00:00,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,I am using the Photoshop's javascript API to f...,How can I find the full path to a font from it...,3040.0,457.0,2008-08-06 03:01:23+00:00,469.0,12.0,<p>Unfortunately the only API that isn't depre...,Unfortunately the only API that isn't deprecat...
3,469,147.0,2008-08-02 15:11:16+00:00,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,I am using the Photoshop's javascript API to f...,How can I find the full path to a font from it...,195170.0,745.0,2008-10-12 07:02:40+00:00,469.0,1.0,<p>There must be a method in Cocoa to get a li...,There must be a method in Cocoa to get a list ...
4,502,147.0,2008-08-02 17:01:58+00:00,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,I have a cross-platform (Python) application w...,Get a preview JPEG of a PDF on Windows?,536.0,161.0,2008-08-02 18:49:07+00:00,502.0,9.0,<p>You can use ImageMagick's convert utility f...,You can use ImageMagick's convert utility for ...
5,502,147.0,2008-08-02 17:01:58+00:00,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,I have a cross-platform (Python) application w...,Get a preview JPEG of a PDF on Windows?,7073.0,878.0,2008-08-10 07:10:19+00:00,502.0,2.0,<p>Is the PC likely to have Acrobat installed?...,Is the PC likely to have Acrobat installed? I ...
6,502,147.0,2008-08-02 17:01:58+00:00,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...,I have a cross-platform (Python) application w...,Get a preview JPEG of a PDF on Windows?,7090.0,13.0,2008-08-10 08:08:33+00:00,502.0,25.0,<p>ImageMagick delegates the PDF->bitmap conve...,ImageMagick delegates the PDF->bitmap conversi...
7,535,154.0,2008-08-02 18:43:54+00:00,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,I'm starting work on a hobby project with a py...,Continuous Integration System for a Python Cod...,538.0,156.0,2008-08-02 18:56:56+00:00,535.0,23.0,<p>One possibility is Hudson. It's written in...,One possibility is Hudson. It's written in Jav...
8,535,154.0,2008-08-02 18:43:54+00:00,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,I'm starting work on a hobby project with a py...,Continuous Integration System for a Python Cod...,541.0,157.0,2008-08-02 19:06:40+00:00,535.0,20.0,"<p>We run <a href=""http://buildbot.net/trac"">B...","We run Buildbot - Trac at work, I haven't used..."
9,535,154.0,2008-08-02 18:43:54+00:00,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...,I'm starting work on a hobby project with a py...,Continuous Integration System for a Python Cod...,660.0,197.0,2008-08-03 12:09:18+00:00,535.0,14.0,<p>Second the Buildbot - Trac inte

In [180]:
merged_df['title_question_answer'] = merged_df['Title'] + merged_df['Body_question'] + merged_df['Body_answer']

Viewing a Sample

In [181]:
print(merged_df['title_question_answer'][1])

How can I find the full path to a font from its display name on a Mac?<p>I am using the Photoshop's javascript API to find the fonts in a given PSD.</p>

<p>Given a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.</p>

<p>This is all happening in a python program running on OSX so I guess I'm looking for one of:</p>

<ul>
<li>Some Photoshop javascript</li>
<li>A Python function</li>
<li>An OSX API that I can call from python</li>
</ul>
<p>I haven't been able to find anything that does this directly.  I think you'll have to iterate through the various font folders on the system: <code>/System/Library/Fonts</code>, <code>/Library/Fonts</code>, and there can probably be a user-level directory as well <code>~/Library/Fonts</code>.</p>



In [238]:
print(merged_df['clean_df'][1])

How can I find the full path to a font from its display name on a Mac?<p>I am using the Photoshop's javascript API to find the fonts in a given PSD.</p>

<p>Given a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.</p>

<p>This is all happening in a python program running on OSX so I guess I'm looking for one of:</p>

<ul>
<li>Some Photoshop javascript</li>
<li>A Python function</li>
<li>An OSX API that I can call from python</li>
</ul>
<p>open up a terminal (Applications-&gt;Utilities-&gt;Terminal) and type this in:</p>

<pre><code>locate InsertFontHere<br></code></pre>

<p>This will spit out every file that has the name you want.</p>

<p>Warning: there may be alot to wade through.</p>


In [217]:
merged_df.head()

,Id_question,OwnerUserId_question,CreationDate_question,Score_question,Title,Body_question,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer,title_question_answer,cleaner_data,Id,Tag,clean_df
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,python,None
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,osx,None
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,fonts,None
3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,photoshop,None
4,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,python,None


In [215]:
print(merged_df['clean_df'][0])

None


In [183]:
print(merged_df['cleaner_data'][1])

How can I find the full path to a font from its display name on a Mac?<p>I am using the Photoshop's javascript API to find the fonts in a given PSD.</p>

<p>Given a font name returned by the API, I want to find the actual physical font file that that font name corresponds to on the disc.</p>

<p>This is all happening in a python program running on OSX so I guess I'm looking for one of:</p>

<ul>
<li>Some Photoshop javascript</li>
<li>A Python function</li>
<li>An OSX API that I can call from python</li>
</ul>
<p>I haven't been able to find anything that does this directly.  I think you'll have to iterate through the various font folders on the system: <code>/System/Library/Fonts</code>, <code>/Library/Fonts</code>, and there can probably be a user-level directory as well <code>~/Library/Fonts</code>.</p>



In [184]:
merged_df = pd.merge(merged_df, tags, left_on='Id_question', right_on='Id', how='left')

In [185]:
merged_df.head()

,Id_question,OwnerUserId_question,CreationDate_question,Score_question,Title,Body_question,Id_answer,OwnerUserId_answer,CreationDate_answer,ParentId,Score_answer,Body_answer,title_question_answer,cleaner_data,Id,Tag
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,python
1,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,osx
2,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,fonts
3,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,497.0,50.0,2008-08-02T16:56:53Z,469.0,4.0,<p>open up a terminal (Applications-&gt;Utilit...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,photoshop
4,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...,518.0,153.0,2008-08-02T17:42:28Z,469.0,2.0,<p>I haven't been able to find anything that d...,How can I find the full path to a font from it...,How can I find the full path to a font from it...,469,python


In [193]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
import spacy
import random

In [187]:
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 10.7 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [188]:
!python -m spacy validate



| Loading compatibility table...
/ Loading compatibility table...
[+] Loaded compatibility table

================= Installed pipeline packages (spaCy v3.6.1) =================
[i] spaCy installation:
C:\Users\15135\AppData\Local\anaconda3\lib\site-packages\spacy

NAME             SPACY            VERSION                              
en_core_web_md   >=3.6.0,<3.7.0   3.6.0   [+]
en_core_web_sm   >=3.6.0,<3.7.0   3.6.0   [+]



In [189]:
import spacy
# Load the installed model "en_core_web_sm"
nlp = spacy.load("en_core_web_sm")

In [190]:
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")


corpus = merged_df['title_question_answer']


def lemmatize(text):
    
    doc = nlp(text.lower())
    lemmas = []
    for token in doc:
        lemmas.append(token.lemma_)
    return ' '.join(lemmas)

# store the review index in the review_idx variable
# either as a random number or a fixed value, e.g. 2557 
review_idx = random.randint(0, len(corpus)-1)

review = merged_df[review_idx]

print('The original text:', review)
print()
print('The lemmatized text:', lemmatize(review))

In [ ]:
raw_df['tokenized_sentences'] = raw_df['sentences'].apply(tokenizer.tokenize)